In [ ]:
from depiction.models.deepbind import DeepBind
from depiction.core import Task, DataType
from depiction.interpreters.uw_model import UWModel

In [ ]:
classifier = DeepBind()

In [ ]:
task = Task.CLASSIFICATION
data_type = DataType.TEXT
class_names = ["NOT BINDING", "BINDING"]

In [ ]:
# Create a LIME tabular interpreter
interpreter = "lime"
explanation_configs = {
    "labels": (1,),
}
interpreter_params = {
    "class_names": class_names,
    "split_expression": list,
    "bow": False,
    "char_level": True
}

explainer = UWModel(interpreter, task, data_type, explanation_configs, **interpreter_params)

In [ ]:
classifier.use_labels = False
explainer.interpret(classifier.predict, "CAAATGTCGAAGTTTA")

In [ ]:
# Create a LIME tabular interpreter
interpreter = "anchor"
explanation_configs = {
    "use_proba": False,
    "batch_size": 100
}
interpreter_params = {
    "class_names": class_names,
    "nlp": classifier.language,
    "unk_token": 'N',
    "sep_token": '',
    "use_unk_distribution": True
}

explainer = UWModel(interpreter, task, data_type, explanation_configs, **interpreter_params)

In [ ]:
classifier.use_labels = True
explainer.interpret(classifier.predict, "TGGCCAACCAGGGGGCGCTT")